In [22]:
# !pip install sagemaker==1.72.0
# !pip install boto3
!pip install -qU -r requirements.txt
# %pip install -qU 'sagemaker>=2.15.0'
!pip list

Package                  Version
------------------------ ------------
abstract_singleton       1.0.1
accelerate               0.20.3
aiohttp                  3.8.4
aiosignal                1.3.1
altair                   4.2.2
aniso8601                9.0.1
anyio                    3.7.0
asttokens                2.2.1
async-generator          1.10
async-timeout            4.0.2
asynctest                0.13.0
attrs                    23.1.0
auto_gpt_plugin_template 0.0.2
autoflake                2.1.1
backcall                 0.2.0
beautifulsoup4           4.12.2
bitsandbytes             0.40.0.post4
black                    23.3.0
blinker                  1.6.2
blis                     0.7.9
boltons                  23.0.0
boto3                    1.14.12
botocore                 1.17.63
brotlipy                 0.7.0
cachetools               5.3.0
catalogue                2.0.8
certifi                  2023.5.7
cffi                     1.15.1
cfgv                     3.3.1
chardet   

In [24]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace

In [26]:

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']
		
hyperparameters = {
	'model_name_or_path':'tiiuae/falcon-40b',
	'output_dir':'/opt/ml/model'
	# add your remaining hyperparameters
	# more info here https://github.com/huggingface/transformers/tree/v4.26.0/examples/pytorch/language-modeling
}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.26.0'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
	entry_point='run_clm.py',
	source_dir='./examples/pytorch/language-modeling',
	instance_type='ml.p3.2xlarge',
	instance_count=1,
	role=role,
	git_config=git_config,
	transformers_version='4.26.0',
	pytorch_version='1.13.1',
	py_version='py39',
	hyperparameters = hyperparameters
)

# starting the train job
huggingface_estimator.fit()

DataNotFoundError: Unable to load data for: endpoints

In [ ]:

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'tiiuae/falcon-40b',
	'SM_NUM_GPUS': json.dumps(4)
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.12xlarge",
	container_startup_health_check_timeout=300,
  )

In [ ]:
# send request
predictor.predict({
	"inputs": "My name is Julien and I like to",
})